In [1]:
#%pip install equinox openmdao[all]

In [2]:
import equinox as eqx
from util import generate_base_model, density, dyn_viscousity
import jax
import jax.numpy as jnp

In [3]:
base_model = generate_base_model()

lift_model = eqx.tree_deserialise_leaves(f"models/lift_surrogate.eqx", base_model)
drag_model = eqx.tree_deserialise_leaves(f"models/drag_surrogate.eqx", base_model)

In [4]:
from optimizer import Optimizer

In [5]:
opt = Optimizer()

In [6]:
opt.load_surrogates(lift_model, drag_model)

In [7]:
opt.define_material(
    E=69e9,
    rho=2700,
    yield_strength=241e6,
    shear_strength=207e6
)

In [8]:
altitude = 10_000

results = opt.solve_wing(
    lift_goal=4e5,
    safety_factor=0.8,
    v_infty=300,
    rho=density(altitude),
    mu=dyn_viscousity(altitude),
    alpha_geo=jnp.deg2rad(jnp.float32(5))
)

Optimizing planform


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were 

Optimization terminated successfully    (Exit mode 0)
            Current function value: 20861.7314797736
            Iterations: 22
            Function evaluations: 53
            Gradient evaluations: 22
Optimization Complete
-----------------------------------
Cl_0 goal: [0.52922581]
Optimizing airfoil
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.004297603687618813
            Iterations: 34
            Function evaluations: 47
            Gradient evaluations: 34
Optimization Complete
-----------------------------------
Optimizing spars
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.12269925770942762
            Iterations: 3
            Function evaluations: 3
            Gradient evaluations: 3
Optimization Complete
-----------------------------------
Main overflow [-0.4]
Rear overflow [-0.0452403]
Spar distance [0.4452403]


In [9]:
results

{'parameters': {'B': array([1.68325042]),
  'T': array([0.15]),
  'P': array([2.79751649]),
  'C': array([0.0202022]),
  'E': array([0.94745157]),
  'R': array([0.003759]),
  'b': array([13.64223256]),
  'c': array([2.248434]),
  'AR': array([6.06743741])},
 'aerodynamics': {'L': Array(400521.71073144, dtype=float64),
  'D': Array(19507.24553156, dtype=float64),
  'Cl_0': array([0.52922581]),
  'alpha_0': Array([-4.82596071], dtype=float64)},
 'structure': {'normal': array([0.50828076]),
  'shear': array([0.3417103]),
  'flange_w': array([0.1]),
  'flange_h': array([0.005]),
  'web_w': array([0.005]),
  'web_h': Array([0.1338784], dtype=float64),
  'spar_ratio': Array([10.50487135], dtype=float64),
  'main_x': array([0.45]),
  'rear_x': array([0.95]),
  'material_usage': array([0.12269926])},
 'timing': {'planform': 75.96692085266113,
  'airfoil': 6.467991590499878,
  'spar': 3.35796856880188}}

In [10]:
{'planform': 21.084315299987793,
  'airfoil': 5.305545330047607,
  'spar': 2.9938952922821045}

{'planform': 21.084315299987793,
 'airfoil': 5.305545330047607,
 'spar': 2.9938952922821045}